In [1]:
import os

In [2]:
%pwd

'c:\\Users\\arbas\\Documents\\Projects\\Machine-Learning\\EndToEnd_Deployed\\Text_Summarization\\research'

In [3]:
os.chdir('../')

In [4]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class Data_Ingestion_Config:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [ ]:
from src.txt_sm.constants import *
from src.txt_sm.utils.common import read_yaml,create_dirs


class Configuration_Manager:
    def __init__(self,config_path=CONFIG_PATH,params_path=PARAMS_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        create_dirs([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> Data_Ingestion_Config:
        config = self.config
        
        create_dirs([config.root_dir])
        data_ingestion_config = Data_Ingestion_Config(root_dir=config.root_dir,
                                                      source_URL=config.source_URL,
                                                      local_data_file=config.local_data_file,
                                                      unzip_dir=config.unzip_dir)
        return data_ingestion_config
    

In [6]:
import urllib.request as request
import zipfile
from src.txt_sm.logging import logger
from src.txt_sm.utils.common import get_size


class DataIngestion:
    def __init__(self,config:Data_Ingestion_Config):
        self.config = config

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as f:
            f.extractall(unzip_path)
                                                     
    def initiate_data_ingestion(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f'{filename} downloaded! with following info \n{headers}')
        else:
            logger.info(f'{filename} already exists! of size {get_size(Path(self.config.local_data_file))}')
        